# Monte Carlo Simulation

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.decomposition import PCA

# Reading the portfolio data into a DataFrame
portfolio = pd.read_csv("portfolio.csv")
returns = pd.read_csv('DailyReturn.csv')
prices = pd.read_csv('DailyPrices.csv')


### Covariance Matrix Function

In [4]:
#give exponential weighted data
def ewCovar(x, lam):
    m, n = x.shape
    w = np.array([(1-lam)*lam**(m-i) for i in range(m)])
    w /= np.sum(w)
    xm = np.mean(x, axis=0)
    x -= xm
    return np.cov(x, aweights=w, rowvar=False)

In [5]:
covar_matrix = ewCovar(returns[portfolio['Stock']].to_numpy(), 0.94)

# Getting the current price from the last row for the stocks in the portfolio
current_prices = prices.iloc[-1][portfolio['Stock']]

KeyError: "['ELV', 'MMC', 'VRTX', 'REGN', 'CB', 'CI', 'ETN', 'SLB', 'PGR', 'BSX'] not in index"

### Simulate Returns with PCA

In [ ]:
n_simulations = 10000
pca = PCA()
pca.fit(covar_matrix)
simulated_returns = pca.transform(np.random.normal(size=(n_simulations, covar_matrix.shape[1])))

In [ ]:
# Cross-joining portfolio with iterations to set up calculation framework
values = pd.merge(portfolio.assign(key=0), pd.DataFrame({'iteration': range(n_simulations)}).assign(key=0), on='key').drop('key', axis=1)

# Calculating current and simulated values, and profit & loss (PnL)
values['current_value'] = values['Holding'] * current_prices[values['Stock']].values
values['simulated_value'] = values['Holding'] * current_prices[values['Stock']].values * (1 + simulated_returns[values['iteration'], values['Stock'].index])
values['pnl'] = values['simulated_value'] - values['current_value']


In [ ]:
# Group by portfolio and iteration and sum up the current value and PnL
portfolio_values = values.groupby(['Portfolio', 'iteration']).agg({'current_value': 'sum', 'pnl': 'sum'}).reset_index()

# Function to calculate VaR
def calculate_var(pnl_series, alpha=0.05):
    return pnl_series.quantile(alpha)

# Group by portfolio and calculate VaR for each
portfolio_risk = portfolio_values.groupby('Portfolio').agg(current_value=('current_value', 'first'), VaR95=('pnl', lambda x: calculate_var(x)))

# Calculate total metrics
total_values = portfolio_values.groupby('iteration').agg({'current_value': 'sum', 'pnl': 'sum'}).reset_index()

# Calculate total VaR
total_risk = total_values.agg(current_value=('current_value', 'first'), VaR95=('pnl', lambda x: calculate_var(x)))
total_risk['Portfolio'] = 'Total'

# Combine portfolio and total risk into one DataFrame
VaR_report = pd.concat([portfolio_risk, total_risk], ignore_index=True)

# Print VaR report
print(VaR_report)


### 